# Board Game Similarity Search

### Use descriptions of Board Games from [Board Game Geek](https://boardgamegeek.com/browse/boardgame) website to find similar board games

### Why?

How often have you found yourself not able to find the right board game for you? This scales to video games, books and movies as well. The proposal here is to use language embedding based similarity search to do a search:

1. For a board game similar to game X.
2. For a board game that makes a specific description. (Finally!)

In [1]:
import pandas as pd
import numpy as np
import similarity_search
from IPython.display import HTML

In [2]:
# top 2000 board games
bg_df = pd.read_csv("bgg_2000.csv")

In [3]:
def pretty_show(game_ranks):
    img_urls = bg_df.loc[game_ranks, 'thumbnail'].to_list()
    game_urls = bg_df.loc[game_ranks, 'url'].to_list()
    return img_urls, game_urls

def get_html_code(img_urls, game_urls):
    html_code ="""
<style>
    img {
        float: left;
        padding: 8px;
    }
</style>
    """
    for img_url, game_url in zip(img_urls, game_urls):
        html_code += """
<a href="%s"><img src="%s" style="width: 300px; height: 300px;"></a>
        """ % (game_url, img_url)

    return html_code

In [4]:
# enter rank - 1 for the selected game
selected_game_rank = 396
results = similarity_search.get_similar_from_game(selected_game_rank, k=9)

game_names = bg_df.loc[results, 'name'].to_list()
print("Games similar to ->", bg_df.loc[selected_game_rank, 'name'])
print('-----')
print('\n'.join(game_names))

Games similar to -> Skull
-----
Abalone
Tussie Mussie
Turn the Tide
Timeline: Events
Timeline: General Interest
Timeline: Discoveries
Timeline: Music & Cinema
Timeline Challenge
Gùgōng


In [5]:
img_urls, game_urls = pretty_show([selected_game_rank])
HTML(get_html_code(img_urls, game_urls))

In [6]:
img_urls, game_urls = pretty_show(results)
HTML(get_html_code(img_urls, game_urls))

In [7]:
import cohere
import os
API_KEY = os.getenv('COHERE_API_KEY')
co = cohere.Client(API_KEY)

In [8]:
def get_embed_from_sentence(sentences):
    response = co.embed(model='small', texts=sentences)
    return response.embeddings

In [9]:
# sentence = "campaign with permanent changes"
# sentence = "two teams card game where a person gives clues"
# sentence = "Esacpe the dungeon with the gold"
sentence = "railway build across United States"
query_embeds = get_embed_from_sentence([sentence])
results = similarity_search.get_similar_from_embeds(query_embeds, 8)
game_names = bg_df.loc[results, 'name'].to_list()
print("Games with descriptions similar to -> \"", sentence,"\"")
print('-----')
print('\n'.join(game_names))

Games with descriptions similar to -> " railway build across United States "
-----
Ride the Rails
Age of Steam
American Rails
Ticket to Ride
Catan Histories: Settlers of America – Trails to Rails
Chicago Express
Whistle Stop
Steam


In [10]:
img_urls, game_urls = pretty_show(results)
HTML(get_html_code(img_urls, game_urls))

### Other websites that can use this

1. Medium and other blog websites
2. NY Times and other newspaper websites
3. Instagram for product placement based on content of a post
4. Highlight a sentence anywhere and search for a specific domain